The purpose of this notebook is to try running MS2LDA on some of our data.

From the [api](http://ms2lda.org/api/) page, it looks like you can run the "decomposition"
step (i.e. mapping given MS2 fragments onto pre-defined motif sets) programmatically.
I can't find info on the available motifsets and how they were defined, but there's one
called "massbank_motifset" which I'm guessing is probably what we want.

In [1]:
# User-defined modules
import os, sys
src_dir = os.path.normpath('../util')
sys.path.insert(0, src_dir)
import util

In [2]:
import pandas as pd

import json
import requests

In [3]:
# Read in the csv data
fname = '../../data/clean/metabolites_and_spectra.csv'
all_mtabs = util.unpackCSV(fname)

This is the example they give:

```python
import requests
import json

spectrum = ('spec_name',188.0818,[(53.0384,331117.7),
(57.0447,798106.4),
(65.0386,633125.7),
(77.0385,5916789.799999999),
(81.0334,27067.0),
(85.0396,740633.6)])

spectra = [spectrum] # or add more to the list

args = {'spectra': json.dumps(spectra), 'motifset': 'massbank_motifset'}

url = 'http://ms2lda.org/decomposition/api/batch_decompose/'

r = requests.post(url,args)
```

In [4]:
# Let's try one spectrum first
for k in all_mtabs:
    if len(all_mtabs[k].MS2) > 1:
        break
all_mtabs[k].MS2[1].peaks


[(136.87, 34.965),
 (180.91, 26.628),
 (198.46, 8.3),
 (198.87, 45.266),
 (254.86, 100.0)]

In [5]:
spectrum = ('test1',0,all_mtabs[k].MS2[1].peaks)
spectra = [spectrum]
args = {'spectra': json.dumps(spectra), 'motifset': 'massbank_motifset'}
url = 'http://ms2lda.org/decomposition/api/batch_decompose/'

r = requests.post(url,args)

In [17]:
r.json()

{u'featureset': u'binned_005',
 u'motifset': u'massbank_motifset',
 u'n_spectra': 1,
 u'result_id': u'310',
 u'result_url': u'http://ms2lda.org/decomposition/api/batch_results/310/',
 u'status': u'SUBMITTED'}

In [24]:
result_id = r.json()['result_id']
url2 = 'http://ms2lda.org/decomposition/api/batch_results/{}/'.format(result_id)
r2 = requests.get(url2)

In [25]:
r2.json()

{u'alpha': [],
 u'decompositions': {u'test1': [[u'motif_22',
    u'motif_22',
    0.003865831650138165,
    0.0,
    u'Water loss  - indicative of a free hydroxyl group \u2013 (in beer often seen in sugary structures)'],
   [u'motif_30',
    u'motif_30',
    0.0036672064937751537,
    0.0,
    u'Fragment indicative for aromatic compounds related to methylbenzene substructure (C7H7 fragment)'],
   [u'motif_6',
    u'motif_6',
    0.003349130443688082,
    0.0,
    u'Amine loss - Indicative for free NH2 group in fragmented molecule'],
   [u'motif_15',
    u'motif_15',
    0.0032736271727618455,
    0.0,
    u'Combined loss of H2O and CO \u2013 indicative for free carboxylic acid group (COOH) \u2013 generic substructure in amino acids and organic acids - this is the second of two motifs (see also motif_20)'],
   [u'motif_2',
    u'motif_2',
    0.0032736271727618455,
    0.0,
    u'Combined loss of H2O and CO \u2013 indicative for free carboxylic acid group (COOH) \u2013 generic substruct

These values are:

'(string:globalmotifname, string:originalmotifname, float:theta, float:overlap_score)'

Okay, now we need to figure out what the theta and overlap_scores means. Which one is equivalent to "amount"?

**Ashvin, task for you**: Figure out how to interpret these cosine scores and theta results. Also, if possible try to figure out what these pre-defined motifsets are - how were they defined? Based on what data? With what data manipulations?

In [27]:
r2.json().keys()

[u'alpha', u'motifset', u'terms', u'decompositions']

In [30]:
r2.json()['motifset']

u'massbank_motifset'

In [31]:
r2.json()['terms']

{u'test1': []}

In [33]:
r2.json()['decompositions'].keys()

[u'test1']

In [38]:
# Turn the decompositions (which is a list of lists) into a dataframe
df = pd.DataFrame(r2.json()['decompositions']['test1'],
                  columns=['motif1', 'motif1_also_i_guess', 'maybe_cosine', 'maybe_abundance', 'annotation'])

# Add a column to label this dataframe with the molecule
df['molecule_name'] = 'hi_ashvin'
df.head()

,motif1,motif1_also_i_guess,maybe_cosine,maybe_abundance,annotation,molecule_name
0,motif_22,motif_22,0.003866,0.0,Water loss - indicative of a free hydroxyl gr...,hi_ashvin
1,motif_30,motif_30,0.003667,0.0,Fragment indicative for aromatic compounds rel...,hi_ashvin
2,motif_6,motif_6,0.003349,0.0,Amine loss - Indicative for free NH2 group in ...,hi_ashvin
3,motif_15,motif_15,0.003274,0.0,Combined loss of H2O and CO – indicative for f...,hi_ashvin
4,motif_2,motif_2,0.003274,0.0,Combined loss of H2O and CO – indicative for f...,hi_ashvin


**Python tip**: you can concatenate pandas dataframes but it's much faster to first put all the dataframes you want to concatenate in a list and then run pd.concat([list, of, dataframes]) (rather than concatenating at each iteration in the list). So I would recommend (1) frun MS2LDA on each MS2, save to file. (2) For each file, read it in as a dataframe, add a column to label the MS2, append to a list. Then (3) Concenate that list into one massive dataframe. And finally, (4) use pd.pivot() to convert your dataframe of concatenated results into a (MS2 X features) table that can be used as input to sklearn functions.

Terminology note: the format of data output from MS2LDA and shown in the dataframe above is "tidy data" (which I love and is awesome), the format of data that sklearn uses (and MATLAB folks tend to be most used to) is wide-form or some other words I can't remember.

This is my favorite post on the matter: https://www.ibm.com/developerworks/community/blogs/jfp/entry/Tidy_Data_In_Python